In [6]:
import sys
import imp
import numpy as np
sys.path.insert(0, '../')

In [7]:
from denoise.graph import io as graphio
imp.reload(graphio)

_, proteins_to_go = graphio.parse_go_label_file("../data/dream_files/human.golabels")
edgelist, node_list, node_map = graphio.parse_graph_file("../data/test_graphs/All_Graphs/Graph1.txt")

In [8]:
from denoise.graph import operations
from denoise.algorithms.dsd import computations
imp.reload(operations)
imp.reload(computations)

A = operations.densify(edgelist)
D = computations.compute_degree_mat(A)
X = computations.compute_X_normalized(A, D)

In [9]:
n, _ = A.shape
labels = {i: proteins_to_go[node_list[i]] for i in range(n)
          if node_list[i] in proteins_to_go}

In [48]:
from denoise import predict
from denoise import scoring
import scipy.spatial.distance as spatial
imp.reload(predict)
imp.reload(scoring)

def create_predictor(E):
    def predictor(training_labels):
        return predict.perform_binary_svc(E, training_labels)
    return predictor

X_scores             = scoring.kfoldcv(5, labels, create_predictor(X))


print("\n10-Nearest Neighbors using DSD embedding:")
print(f"KNN on raw network: {np.mean(X_scores) * 100}%")


10-Nearest Neighbors using DSD embedding:
KNN on raw network: 0.0%
